In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.utils import to_categorical


def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential()

   
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))

   
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dropout(0.5))

     Fully Connected
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

   
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def load_example_data():
   
    num_samples = 100 
    num_frames = 30    
    height, width, channels = 64, 64, 3  
    num_classes = 10   

    X_train = np.random.rand(num_samples, num_frames, height, width, channels) 
    y_train = np.random.randint(0, num_classes, num_samples) 
    X_test = np.random.rand(num_samples // 2, num_frames, height, width, channels)  
    y_test = np.random.randint(0, num_classes, num_samples // 2)  

   
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = load_example_data()


input_shape = (None, 64, 64, 3)  
num_classes = 10


model = create_cnn_lstm_model(input_shape, num_classes)


history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


model.save('sign_language_cnn_lstm.h5')


model = load_model('sign_language_cnn_lstm.h5')


def prepare_test_sample(num_frames=30, height=64, width=64, channels=3):
    
    test_sample = np.random.rand(1, num_frames, height, width, channels)
    return test_sample

X_new = prepare_test_sample()


predictions = model.predict(X_new)
predicted_class = np.argmax(predictions)

print("Predicted Class:", predicted_class)


for i, prob in enumerate(predictions[0]):
    print(f"Class {i}: {prob:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 99s 21s/step - accuracy: 0.0787 - loss: 2.3781 - val_accuracy: 0.1200 - val_loss: 2.3263
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 69s 15s/step - accuracy: 0.1465 - loss: 2.3358 - val_accuracy: 0.1200 - val_loss: 2.3190
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 15s/step - accuracy: 0.0992 - loss: 2.4146 - val_accuracy: 0.1200 - val_loss: 2.3131
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 82s 15s/step - accuracy: 0.1138 - loss: 2.3868 - val_accuracy: 0.1200 - val_loss: 2.3101
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 65s 15s/step - accuracy: 0.1508 - loss: 2.3161 - val_accuracy: 0.1200 - val_loss: 2.3137
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 15s/step - accuracy: 0.0928 - loss: 2.3726 - val_accuracy: 0.1200 - val_loss: 2.3097
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 15s/step - accuracy: 0.1015 - loss: 2.4110 - val_accuracy: 0.1200 - val_loss: 2.3012
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 15s/step - accuracy: 0.0798 - loss: 2.2888 - val_accuracy: 0.1200 - val_loss: 2.2949
